In [13]:
import pandas as pd
from variables_dict_builder import variables_dict_builder

variables_dict = variables_dict_builder()
variables_suffixes = [*variables_dict.keys()]

df = (pd.read_csv('../intermediate_data/stacked_filtered_data.csv')
      .apply(pd.to_numeric, errors='coerce')
      )

In [14]:
# variable finder
variable_suffix = "cohbyr"
[col for col in df.columns if variable_suffix in col]

['s1cohbyr',
 's2cohbyr',
 's3cohbyr',
 's4cohbyr',
 's5cohbyr',
 's6cohbyr',
 's7cohbyr',
 's8cohbyr',
 's9cohbyr',
 's10cohbyr',
 's11cohbyr',
 's12cohbyr',
 's13cohbyr',
 's14cohbyr',
 's15cohbyr',
 'racohbyr']

In [15]:
## WIDE TO LONG OF SELF/SPOUSE VARIABLES
ADL_suffixes = ['batha','dressa','eata','beda','toilta','walkra','walk1a','walksa', #ADLs
                    'shopa','phonea','moneya','mealsa','medsa','mapa', #IADLs
                    'clim1a','climsa','chaira','stoopa','lifta','armsa','dimea','pusha','sita', # Other functional limitations
                    ]

melt_suffixes = ['nrshom', 'nhmliv', #outcomes
                 'agey_e',
                 'mstat',
                 'urbrur',
                 'livsib', 'momage', 'dadage',
                 'arthre','cancre','diabe','hearte','hibpe','lunge','psyche','stroke', #Diagnoses
                 'bmi', 'smokev'
                 ] + ADL_suffixes

# Pre-creating a dataframe with household ID X wave to join all of our wide to long outputted variables to
waves = list(range(1, 16))
hhidpn_waves = pd.MultiIndex.from_product([df['hhidpn'].unique(), waves], names=['hhidpn', 'wave'])
final_df = pd.DataFrame(index=hhidpn_waves).reset_index()

for suffix in melt_suffixes:
    # Generate the self and spouse columns for the current suffix
    self_columns = [f'r{i}{suffix}' for i in range(1, 16)]
    spouse_columns = [f's{i}{suffix}' for i in range(1, 16)]
    
    if suffix in ADL_suffixes:
        self_columns = self_columns[1:]; spouse_columns = spouse_columns[1:]
    
    # Melt the self for this suffix
    self_df = df.melt(id_vars=['hhidpn'], value_vars=self_columns,
                          var_name='wave', value_name=f'{suffix}_self')
    self_df['wave'] = self_df['wave'].str.extract('(\d+)').astype(int)
    
    # Melt the spouse for this suffix
    spouse_df = df.melt(id_vars=['hhidpn'], value_vars=spouse_columns,
                        var_name='wave', value_name=f'{suffix}_spouse')
    spouse_df['wave'] = spouse_df['wave'].str.extract('(\d+)').astype(int)

    
    merged_df = pd.merge(self_df, spouse_df, on=['hhidpn', 'wave'])
    
    final_df = pd.merge(final_df, merged_df[['hhidpn', 'wave', f'{suffix}_self', f'{suffix}_spouse']], 
                        on=['hhidpn', 'wave'], how='left')
    
final_df = final_df.sort_values(by=['hhidpn', 'wave'])

In [16]:
## WIDE TO LONG OF HOUSEHOLD VARIABLES
household_suffixes = ['hhres','child','atotn', 'itot', 'inpov']

for suffix in household_suffixes:
    hh_columns = [f'h{i}{suffix}' for i in range(1, 16)]
    
    # Melt the self for this suffix
    hh_df = df.melt(id_vars=['hhidpn'], value_vars=hh_columns,
                          var_name='wave', value_name=f'{suffix}_household')
    hh_df['wave'] = hh_df['wave'].str.extract('(\d+)').astype(int)
    
    final_df = pd.merge(final_df, hh_df[['hhidpn', 'wave', f'{suffix}_household']], 
                        on=['hhidpn', 'wave'], how='left')

In [17]:
## JOINING OF WAVE PERSISTENT VARIABLES
wave_persistent_variables = ['racohbyr','radyear','raracem', 'ragender', 'rahispan', 'rarelig', 'ravetrn', 'raedyrs', 'raedegrm', 'raeduc', 'rameduc', 'rafeduc']
final_df = pd.merge(final_df, df[['hhidpn'] + wave_persistent_variables],
                    on=['hhidpn'], how='left')

In [18]:
# Removing observations once they're dead
import numpy as np

vars_to_check_na = final_df.columns.difference(["hhidpn", "wave"] + wave_persistent_variables)
final_df = final_df.loc[np.logical_not(final_df[vars_to_check_na].isna().all(axis=1))]

In [19]:
# Construction of Outcome Y:="next year nursing home entry" and dropping every observation after they enter nursing home or die
final_df["nhmliv_self_next_wave"] = final_df.groupby("hhidpn")["nhmliv_self"].shift(-1)

def mask_group(group):
    first_one_idx = group[group["nhmliv_self_next_wave"] == 1].index.min()  # Find the index of the first occurrence of nursing home entrance next wave
    if pd.isna(first_one_idx):  # If there are no 1s, keep all rows
        return group
    return group.loc[:first_one_idx]  # Keep rows up to and including the first occurrence of var_a = 1

# Apply the mask to each person_id group
final_df = final_df.groupby("hhidpn", group_keys=False).apply(mask_group)

final_df = final_df.dropna(subset=["nhmliv_self_next_wave"])

# 2963 of 41772 hhidpns enter a nursing home

In [27]:
pd.set_option('display.max_columns', 200)
final_df[['hhidpn','wave','child_household']]

,hhidpn,wave,child_household
16,2010,2,8.0
17,2010,3,8.0
18,2010,4,9.0
31,3010,2,5.0
32,3010,3,5.0
...,...,...,...
636069,959738010,10,0.0
636070,959738010,11,1.0
636071,959738010,12,1.0
636072,959738010,13,1.0


In [21]:
## One hot encoding for categoricals

final_df['rafemale'] = final_df['ragender'] - 1
final_df['married_self'] = final_df['mstat_self'].isin([1, 3]).astype(int)
final_df['urban_self'] = final_df['urbrur_self'].isin([1]).astype(int)
final_df['rural_self'] = final_df['urbrur_self'].isin([3]).astype(int)
final_df['rablack'] = final_df['raracem'].isin([2]).astype(int)

for cohort in range(8):
    final_df[f'racohbyr_{cohort}'] = final_df['racohbyr'].isin([cohort]).astype(int)

In [22]:
# Interaction and quadratic terms
final_df['agey_e_self_squared'] = final_df['agey_e_self']**2
final_df['raedyrs_squared'] = final_df['raedyrs']**2

final_df['agey_e_self_squared_X_mstat'] = final_df['agey_e_self_squared'] * final_df['mstat_self']
final_df['agey_e_self_squared_X_female'] = final_df['agey_e_self_squared'] * final_df['rafemale']
final_df['agey_e_self_X_raedyrs'] = final_df['agey_e_self'] * final_df['raedyrs']

In [29]:
# drop wave 1 and save final cleaned data
final_df = final_df[final_df['wave'] > 1]

final_df[['hhidpn','wave','nhmliv_self_next_wave',
          'rafemale','rablack','rahispan','ravetrn','raedyrs','rameduc','rafeduc',
          'racohbyr_1','racohbyr_2','racohbyr_3','racohbyr_4','racohbyr_5','racohbyr_6','racohbyr_7', #'racohbyr_0', leave one out
          'agey_e_self','married_self','urban_self','rural_self',
          'atotn_household', 'itot_household', 'child_household', 'inpov_household',
          'livsib_self', 'momage_self', 'dadage_self',
          'arthre_self', 'cancre_self', 'diabe_self', 'hearte_self', 'hibpe_self', 'lunge_self', 'psyche_self', 'stroke_self', 'bmi_self', 'smokev_self', 'batha_self', 'dressa_self', 'eata_self', 'beda_self', 'toilta_self', 'walkra_self', 'walk1a_self', 'walksa_self', 'shopa_self', 'phonea_self', 'moneya_self', 'mealsa_self', 'medsa_self', 'mapa_self', 'clim1a_self', 'climsa_self', 'chaira_self', 'stoopa_self', 'lifta_self', 'armsa_self', 'dimea_self', 'pusha_self', 'sita_self',
          'agey_e_self_squared','raedyrs_squared','agey_e_self_squared_X_mstat','agey_e_self_squared_X_female','agey_e_self_X_raedyrs'
          ]].to_csv('../final_data/clean_model_data.csv', index=False)